In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from pandas import ExcelWriter

import numpy as np
import datetime as dt

import re

In [2]:
# name the output file for the tables
tables_out = "/home/gus/MEGAsync/zim/main/Yale/Projects/G_pallidipes/Reports/2015_grant_report/Sampling_Map_and_Tables/Sampling_Tables/crosssectional_tables_draft_2.xls"

table_writer = ExcelWriter(tables_out)

In [3]:
main_table = pd.read_excel('/home/gus/Documents/YalePostDoc/project_stuff/G_pallidipes_kenya/gpd_combined_datatable_refined_2015-09-26_KENYA.xls')

In [4]:
yyyy = re.compile(r'\b[12]\d\d\d\b')
def get_YEAR(x):
    
    unknown = 'Unk.'
    
    if isinstance(x,dt.datetime):
        return x.year
    
    if str(x) == 'nan':
        return unknown
    
    if isinstance(x,(str,unicode)):
        matches = yyyy.findall(x)
        if len(matches) == 0:
            return unknown
        else:
            return int(matches[0])
    else:
        raise ValueError

In [5]:
# recode DATE to be string of year or Unk.

main_table["YEAR"] = main_table.DATE.apply(get_YEAR)
main_table[["YEAR","DATE"]][18:21]

,YEAR,DATE
18,Unk.,NaN
19,Unk.,NaN
20,2003,2003-04-25 00:00:00


In [6]:
# recode SEX so NaN becomes string rather than Not_a_Number object 
# and questioned values like 'M (?)' are explicitly set to UNKNOWN

main_table.SEX = main_table.SEX.apply(lambda x: 'Unk.' if ((str(x) == 'nan') or '?' in x)  else x )
main_table.head(2)

,RECORD_SOURCE,LOCATION_LONG,SITE_SUB_GROUP,SITE_GROUP,COUNTRY,TRAP_NUMBER,DATE,SPECIES,SEX,TENERAL,DEAD,FLY_NUMBER,HUNGER_STAGE,WING_FRAY,PROBOSCIS_TRYPS,MIDGUT_TRYPS,SALIVARY_GLAND_TRYPS,KEPT_IN,KEPT_IN_SIMPLE,COMMENT,TUBE_OR_BOX,METHOD_OF_PREPARATION,BOX_NUMBER_ID,SOURCE,LATITUDE,LONGITUDE,ELEVATION,INFECTION,TISSUE,TISSUE_OR_DNA,DERIVATIVE,YEAR
0,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,1,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.
1,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,2,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.


In [7]:
# get table of just Gpd flies
gpd = main_table.query("SPECIES == 'Gpd'")

In [8]:
# get Gpd with no question about site_group
gpd_noQ = gpd[gpd.SITE_GROUP.apply(lambda x: True if "?" not in x else False)]
# gpd_noQ.head()

# Kenya overall: 

### Remove brian's 2015 data from THIS data and number of SITES that make up SITE_GROUP

In [9]:
def get_SITE_GROUP_Ns(x, df):
    site_group = """ SITE_GROUP == "{x}" """.format(x=x)
    sites_len = len(kenya_final.query(site_group)['SITE_SUB_GROUP'].unique())
    return """{x} ({sites_len})""".format(x=x, sites_len=sites_len)

In [10]:
kenya_final = gpd_noQ.query(''' YEAR != (2015,'Unk.') ''').copy()

In [11]:
kenya_final['SITE_GROUP_N'] = kenya_final.SITE_GROUP.apply(lambda x: get_SITE_GROUP_Ns(x=x,df=kenya_final))

In [17]:
YEAR_TISSUE_OR_DNA = \
pd.crosstab(index=kenya_final.SITE_GROUP_N, 
            columns=[kenya_final.YEAR,kenya_final.TISSUE_OR_DNA], 
            values=None, 
            rownames=None, colnames=None, 
            aggfunc=None, 
            margins=True, 
            dropna=True)

# YEAR_TISSUE_OR_DNA.to_excel(table_writer,sheet_name="YEAR_TISSUE_OR_DNA")
# table_writer.save()
YEAR_TISSUE_OR_DNA

YEAR                    2000 2001 2002  2003 2009          2010   All
TISSUE_OR_DNA            DNA  DNA  DNA   DNA  DNA Tissue Tissue      
SITE_GROUP_N                                                         
Galana (2)               120    0    0     0    0      0      0   120
Kabunkanga (1)             0    0    0     0    0      0     14    14
Kapesur (1)                0    0    0     0  102     41      0   143
Kathekani (1)             56    0    0     0    0      0      0    56
Ketri Insectary (1)        0    0    0    92    0      0      0    92
Kwale (1)                  0    0    0   466    0      0      0   466
Lambwe Valley (5)         49    0  199   154    0      0      0   402
Narok (1)                  0    0    0    61    0     28      0    89
Nguruman (9)              75   14   50  1371    0     77      0  1587
Ruma National Park (13)    0    0    0   995    0      0      0   995
All                      300   14  249  3139  102    146     14  3964

# Nguruman specifically: 

In [13]:
nguruman_final = gpd_noQ.query(''' YEAR != 22222 ''').query(''' SITE_GROUP == 'Nguruman' ''').copy()

In [14]:
nguruman_final.head()

,RECORD_SOURCE,LOCATION_LONG,SITE_SUB_GROUP,SITE_GROUP,COUNTRY,TRAP_NUMBER,DATE,SPECIES,SEX,TENERAL,DEAD,FLY_NUMBER,HUNGER_STAGE,WING_FRAY,PROBOSCIS_TRYPS,MIDGUT_TRYPS,SALIVARY_GLAND_TRYPS,KEPT_IN,KEPT_IN_SIMPLE,COMMENT,TUBE_OR_BOX,METHOD_OF_PREPARATION,BOX_NUMBER_ID,SOURCE,LATITUDE,LONGITUDE,ELEVATION,INFECTION,TISSUE,TISSUE_OR_DNA,DERIVATIVE,YEAR
0,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,1,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.
1,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,2,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.
2,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,3,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.
3,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,4,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.
4,Not Sure,"Nguruman, Kattekani, Kenya",Kattekani,Nguruman,KENYA,NONE,NaN,Gpd,Unk.,NaN,NaN,5,NaN,NaN,NaN,NaN,NONE,NONE,Unk.,NaN,box,NONE,3,-80 Freezer,NONE,NONE,NONE,NONE,NONE,DNA,gDNA,Unk.


In [16]:
NGURUMAN_YEAR_TISSUE_OR_DNA = \
pd.crosstab(index=nguruman_final.SITE_SUB_GROUP, 
            columns=[nguruman_final.YEAR,nguruman_final.TISSUE_OR_DNA], 
            values=None, 
            rownames=None, colnames=None, 
            aggfunc=None, 
            margins=True, 
            dropna=True)

# NGURUMAN_YEAR_TISSUE_OR_DNA.to_excel(table_writer,sheet_name="NGURUMAN_YEAR_TISSUE_OR_DNA")
# table_writer.save()
NGURUMAN_YEAR_TISSUE_OR_DNA

YEAR,2000,2001,2002,2003,2009,2015,Unk.,All
TISSUE_OR_DNA,DNA,DNA,DNA,DNA,Tissue,Tissue,DNA,
SITE_SUB_GROUP,,,,,,,,
Camp,0,0,0,48,0,0,0,48
Campsite,0,0,0,130,0,5,24,159
Kalema,18,14,0,15,0,0,18,65
Kattekani,0,0,0,0,0,0,5,5
Lengobei,0,0,0,94,0,0,0,94
Lengongu,0,0,0,142,0,48,36,226
Mukinyo,0,0,0,396,0,50,0,446
Nguruman,0,0,0,7,77,0,53,137
